# 可视化与扩散动图导出
本 Notebook 演示如何调用 `generate_denoise_animation.py` 中的工具函数，生成扩散过程帧并构建命令行。

## 1. 自动定位项目根目录
保证后续导入脚本、读取配置时不会出现路径问题。

In [ ]:
from pathlib import Path
import sys

def 找到项目根目录(起点: Path) -> Path:
    for parent in [起点, *起点.parents]:
        if (parent / "generate_denoise_animation.py").exists():
            return parent
    raise FileNotFoundError("未定位到项目根目录，请确认 Notebook 位于仓库内部。")

PROJECT_ROOT = 找到项目根目录(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("项目根目录:", PROJECT_ROOT)

## 2. 计算合适的帧步长
`adjust_stride` 会根据扩散总步数与帧数上限自动调整间隔，避免动图帧数过多。

In [ ]:
from generate_denoise_animation import adjust_stride

总步数 = 1000
步长 = adjust_stride(总步数, stride=1, max_frames=60)
print(f"扩散总步数 {总步数}，推荐的帧步长为: {步长}")

## 3. 将张量序列转换为图像
使用 `tensors_to_images` 将扩散过程中的张量帧转换为 PIL 图像，便于保存或制作动图。

In [ ]:
import torch
from generate_denoise_animation import tensors_to_images

模拟帧 = [(t, torch.rand(1, 1, 16, 16)) for t in range(10, -1, -1)]

图像序列 = tensors_to_images(
    模拟帧,
    sample_index=0,
    use_jet=True,
    threshold=None,
    compress_factor=None,
    tail_frames=3,
)

print(f"共生成 {len(图像序列)} 帧，首帧类型: {type(图像序列[0][1])}")

## 4. 构造动图导出命令
最后生成一条可直接在终端执行的命令，用于导出扩散动图。

In [ ]:
def 构造动图命令(config: Path, lr_image: str, sampler: str = "ddim") -> str:
    return (
        f"python generate_denoise_animation.py --config {config.as_posix()} "
        f"--lr-image {lr_image} --sampler {sampler}"
    )

命令 = 构造动图命令(PROJECT_ROOT / "configs/eval.json", lr_image="example.png")
print("动图生成命令:", 命令)

## 5. 使用建议
- 在生成真实动图前，请确认 `configs/eval.json` 指向正确的评估数据与模型权重；
- 推荐在项目根目录终端执行上述命令，这样输出路径与依赖更容易管理；
- 可根据需要调整 `--frame-stride`、`--max-frames`、`--fps` 等参数。